In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import holidays

from sklearn.model_selection import (
    train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [2]:
def save_results_to_csv(data, base_filename):
    """
    Converts the input data to a DataFrame (if not already) and saves it to a CSV file with the 
    current date and time appended to the filename. Automatically prints the filename of the saved CSV file.

    Parameters:
    data: Data to be saved, can be a DataFrame, dictionary, list of lists, or a NumPy array.
    base_filename (str): Base filename without extension.
    """
    # Convert the input data to a DataFrame if it's not already one
    if not isinstance(data, pd.DataFrame):
        df = pd.DataFrame(data)
    else:
        df = data

    # Get the current date and time
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Construct filename with date, time, and .csv extension
    filename = f"{base_filename}_{current_time}.csv"

    # Save DataFrame to CSV
    df.to_csv(filename, index=False)

    # Print the filename for reference
    print(f"Results saved as {filename}")


def save_submission_csv(test_data, predictions, model_name):
    """
    Save model predictions to a CSV file with a formatted filename.

    Args:
        test_data (pd.DataFrame): The test data.
        predictions (pd.Series or np.array): Model predictions for the test data.
        model_name (str): The name of the model.

    Returns:
        None
    """
    # Create a dictionary for storing results with Ids and predictions
    results_dict = {'Id': np.arange(test_data.shape[0]), 'log_bike_count': predictions}

    # Convert the dictionary to a DataFrame
    results_df = pd.DataFrame(results_dict)

    # Format the submission CSV filename with model name, date, and time
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    submission_filename = f"submission_{model_name}_{current_datetime}.csv"

    # Save to CSV
    results_df.to_csv(submission_filename, index=False)

    # Print the first few rows of the results to check
    print(f"Saved submission to {submission_filename}")
    print(results_df.head())    
    
    
    
def preprocess_data(train_file, test_file, final_test_file, weather_file, lockdown_file, holiday_file,
                    subscribers_file, sncf_file):
    # Load main datasets
    train_data = pd.read_parquet(train_file)
    test_data = pd.read_parquet(test_file)
    final_test_data = pd.read_parquet(final_test_file)
    
    # Combining datasets
    combined_train_test = pd.concat([train_data, test_data], axis=0)
    combined_train_test.dropna(inplace=True)  
    train_data = combined_train_test  
    test_data = final_test_data

    # Load external datasets
    weather_data = pd.read_csv(weather_file)
    lockdown_data = pd.read_csv(lockdown_file)
    holiday_data = pd.read_csv(holiday_file)
    velib_subscribers = pd.read_csv(subscribers_file)
    sncf_passengers_delayed = pd.read_csv(sncf_file)

    # Standardize and merge date columns
    standardize_date_column(train_data, test_data, final_test_data, weather_data, 
                            lockdown_data, holiday_data, velib_subscribers, sncf_passengers_delayed)

    # Filter necessary columns from weather_data and lockdown_data
    weather_data = weather_data[['date', 'feelslike', 'humidity', 'precip', 'windspeed']]
    lockdown_data = lockdown_data[['date', 'school_closures', 'full_lockdown']]

    # Merge with external data
    external_datasets = [holiday_data, weather_data, lockdown_data, 
                         velib_subscribers, sncf_passengers_delayed]
    train_data = merge_all_external_data(train_data, external_datasets)
    test_data = merge_all_external_data(test_data, external_datasets)

    # Apply date encoding and temperature binning
    train_data = _encode_dates(train_data)
    train_data = bin_temperature(train_data)
    test_data = _encode_dates(test_data)
    test_data = bin_temperature(test_data)

    # Remove outliers from train data
    train_data = remove_outliers(train_data, 'log_bike_count')

    # Drop specified columns
    columns_to_drop = ['counter_id', 'counter_installation_date', 'counter_technical_id', 'coordinates', 'site_id', 
                       'site_name', 'latitude', 'longitude', 'bike_count']
    train_data.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    test_data.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    

    # Ensure test data has all columns from train data, except the target variable
    for column in train_data.columns:
        if column not in test_data.columns and column != 'log_bike_count':
            test_data[column] = 0

    # Align columns in test data to match train data
    test_data = test_data[train_data.columns.drop('log_bike_count')] 

    return train_data, test_data

def merge_dataframes_on_columns(df1, df2, join_columns):
    """
    Merge two dataframes on specified columns while maintaining the order of the first dataframe.
    
    :param df1: The first dataframe.
    :param df2: The second dataframe to merge.
    :param join_columns: List of column names on which to join.
    :return: Merged dataframe.
    """
    merged_df = pd.merge(df1, df2, on=join_columns, how='left')
    return merged_df

def standardize_date_column(*dataframes):
    for df in dataframes:
        if 'datetime' in df.columns:
            df.rename(columns={'datetime': 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

def merge_all_external_data(main_data, external_datasets):
    for dataset in external_datasets:
        main_data = _merge_external_data(main_data, dataset, 'date')
        main_data.drop(columns=['date_y'], inplace=True, errors='ignore')
    return main_data

def _merge_external_data(X, df_ext, on_column):
    X = X.copy()
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(X.sort_values('date'), df_ext.sort_values('date'), left_on='date', right_on='date')
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X


def _encode_dates(X):
    X = X.copy()
    fr_holidays = holidays.France()  # Get the holiday calendar for France

    # Extract date components
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    X["week"] = X["date"].dt.isocalendar().week

    # Determine if the date is a French holiday
    X["is_holiday"] = X["date"].apply(lambda d: d in fr_holidays).astype(int)

    # Cosine and sine encodings for hours, months, and weekdays
    X["hour_sin"] = np.sin(2 * np.pi * X["hour"] / 23.0)
    X["hour_cos"] = np.cos(2 * np.pi * X["hour"] / 23.0)
    X["weekday_sin"] = np.sin(2 * np.pi * X["weekday"] / 6.0)
    X["weekday_cos"] = np.cos(2 * np.pi * X["weekday"] / 6.0)
    X["month_sin"] = np.sin(2 * np.pi * X["month"] / 11.0)
    X["month_cos"] = np.cos(2 * np.pi * X["month"] / 11.0)

    # Season encoding
    X["season"] = X["month"].apply(lambda m: (m % 12 + 3) // 3)

    # Rush hour for weekdays not on holidays
    X["morning_rush"] = ((X["weekday"] < 5) & (X["hour"] >= 7) & (X["hour"] <= 9) & (X["is_holiday"] == 0)).astype(int)
    X["evening_rush"] = ((X["weekday"] < 5) & (X["hour"] >= 16) & (X["hour"] <= 18) & (X["is_holiday"] == 0)).astype(int)

    # One-hot encode year and weekday
    year_dummies = pd.get_dummies(X['year'], prefix='year')
    weekday_dummies = pd.get_dummies(X['weekday'], prefix='weekday')

    # Concatenate with original DataFrame
    X = pd.concat([X, year_dummies, weekday_dummies], axis=1)

    # Drop original date components
    X.drop(columns=['year', 'month', 'day', 'weekday', 'hour', 'week', 'date'], inplace=True)
    
    return X



def bin_temperature(df):
    bins = [-float('inf'), 10, 20, 25, float('inf')]
    labels = ['cold', 'cool', 'warm', 'hot']
    df['temp_binned'] = pd.cut(df['feelslike'], bins=bins, labels=labels)

    # One-hot encode the binned temperatures
    temp_dummies = pd.get_dummies(df['temp_binned'], prefix='temp')
    df = pd.concat([df, temp_dummies], axis=1)

    # Drop the original binned column and feelslike
    df.drop(columns=['temp_binned', 'feelslike'], inplace=True)
    return df


def remove_outliers(df, column, multiplier=1.5):
    """
    Remove outliers from a dataframe based on the IQR in a specified column.

    :param df: DataFrame to process.
    :param column: The name of the column to check for outliers.
    :param multiplier: The multiplier for the IQR to define what is considered an outlier.
    :return: DataFrame with outliers removed.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR

    return df[~((df[column] < lower_bound) | (df[column] > upper_bound))]

# Function to perform K-Fold Cross Validation and calculate RMSE
def perform_kfold_cv_rmse(model, X, y, kf):
    rmse_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        rmse = math.sqrt(mse)
        rmse_scores.append(rmse)

    return rmse_scores

In [3]:
train_data_processed, test_data_processed = preprocess_data(
    "train.parquet", "test.parquet", "final_test.parquet", 
    "hourly-weather-data.csv", "lockdown-data.csv", "paris_school_holidays_2020_2022_correct.csv",
    "velib_subscribers_2020_2022.csv", "sncf_passengers_delayed_hourly_2020_2022.csv"
)

In [4]:
train_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496827 entries, 107 to 496826
Data columns (total 35 columns):
 #   Column                               Non-Null Count   Dtype   
---  ------                               --------------   -----   
 0   counter_name                         496827 non-null  category
 1   log_bike_count                       496827 non-null  float64 
 2   is_school_holiday                    496827 non-null  int64   
 3   humidity                             496827 non-null  float64 
 4   precip                               496827 non-null  float64 
 5   windspeed                            496827 non-null  float64 
 6   school_closures                      496827 non-null  int64   
 7   full_lockdown                        496827 non-null  int64   
 8   Subscribers                          496827 non-null  int64   
 9   SNCFpassengersDelayedInParisPerHour  496827 non-null  float64 
 10  is_holiday                           496827 non-null  int32   
 11

In [5]:
for col in test_data_processed :
    print(col)

counter_name
is_school_holiday
humidity
precip
windspeed
school_closures
full_lockdown
Subscribers
SNCFpassengersDelayedInParisPerHour
is_holiday
hour_sin
hour_cos
weekday_sin
weekday_cos
morning_rush
evening_rush
year_2020
year_2021
month_1
month_2
month_3
month_4
month_5
month_6
month_7
month_8
month_9
month_10
month_11
month_12
temp_cold
temp_cool
temp_warm
temp_hot


In [6]:
# splitting the X y from the training data 
X = train_data_processed.drop('log_bike_count', axis=1)
y = train_data_processed['log_bike_count']

# Splitting the data
X_base, X_meta, y_base, y_meta = train_test_split(X, y, test_size=0.2, random_state=42)

# Define KFold for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [7]:
# counter_name is the only categorical column
categorical_features = ['counter_name']
numerical_features = ["Subscribers", "SNCFpassengersDelayedInParisPerHour", "humidity", "precip", "windspeed"]

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

### Model 1): Catboost

In [8]:
# Define the index of the categorical feature
cat_features_index = [X_base.columns.get_loc('counter_name')]

# Initialize the CatBoost model with hyperparameters obtained through gridsearch
catboost_model = CatBoostRegressor(
    depth=10,
    iterations=1000,
    rsm=0.25,
    sampling_frequency="PerTree",
    subsample=0.7,
    cat_features=cat_features_index,
    l2_leaf_reg=3,  # Regularization
    verbose=100
)
# Perform K-Fold Cross Validation and calculate RMSE
cv_rmse_scores = perform_kfold_cv_rmse(catboost_model, X_base, y_base, kf)

# Output the RMSE for each fold
print("K-Fold Cross-Validation RMSE Scores:", cv_rmse_scores)
print("Average RMSE:", np.mean(cv_rmse_scores))


0:	learn: 1.6278756	total: 180ms	remaining: 3m
100:	learn: 0.7031123	total: 7.25s	remaining: 1m 4s
200:	learn: 0.6190200	total: 15s	remaining: 59.5s
300:	learn: 0.5700312	total: 23s	remaining: 53.4s
400:	learn: 0.5385035	total: 31.3s	remaining: 46.7s
500:	learn: 0.5184588	total: 39.7s	remaining: 39.6s
600:	learn: 0.5023233	total: 48.1s	remaining: 31.9s
700:	learn: 0.4890522	total: 56.5s	remaining: 24.1s
800:	learn: 0.4792595	total: 1m 4s	remaining: 16.1s
900:	learn: 0.4705094	total: 1m 13s	remaining: 8.06s
999:	learn: 0.4634919	total: 1m 22s	remaining: 0us
0:	learn: 1.6293118	total: 69.6ms	remaining: 1m 9s
100:	learn: 0.7031230	total: 8.44s	remaining: 1m 15s
200:	learn: 0.6169299	total: 16.9s	remaining: 1m 7s
300:	learn: 0.5731581	total: 25.3s	remaining: 58.8s
400:	learn: 0.5417602	total: 33.7s	remaining: 50.4s
500:	learn: 0.5180124	total: 42.9s	remaining: 42.7s
600:	learn: 0.5011585	total: 51.8s	remaining: 34.4s
700:	learn: 0.4871206	total: 1m 2s	remaining: 26.5s
800:	learn: 0.4773677

In [9]:
# Retrieve feature importance
feature_importances = catboost_model.get_feature_importance()

# Get feature names
feature_names = X_base.columns

# Combine feature names and their importance scores
features_and_importance = zip(feature_names, feature_importances)

# Sort the features by importance
sorted_features_and_importance = sorted(features_and_importance, key=lambda x: x[1], reverse=True)

# Print the feature importance
print("Feature Importances:")
for feature, importance in sorted_features_and_importance:
    print(f"{feature}: {importance:.2f}")

Feature Importances:
hour_cos: 31.63
counter_name: 20.70
hour_sin: 17.00
weekday_sin: 3.60
temp_cold: 3.31
morning_rush: 2.16
humidity: 2.13
weekday_cos: 1.97
full_lockdown: 1.45
is_school_holiday: 1.43
evening_rush: 1.18
month_9: 1.10
month_2: 1.07
windspeed: 1.03
month_5: 0.95
SNCFpassengersDelayedInParisPerHour: 0.90
month_6: 0.85
month_7: 0.78
month_10: 0.63
temp_cool: 0.62
month_1: 0.61
temp_warm: 0.59
precip: 0.52
is_holiday: 0.52
year_2020: 0.44
Subscribers: 0.41
year_2021: 0.40
month_8: 0.38
school_closures: 0.36
month_11: 0.31
month_12: 0.30
month_4: 0.29
temp_hot: 0.23
month_3: 0.16


In [10]:
# Saving the results in a CSV
save_results_to_csv(sorted_features_and_importance, "feature_importances")

Results saved as feature_importances_20231209_075536.csv


In [ ]:
# Define the hyperparameter space for Bayesian optimization
space = { 
    'depth': hp.choice('depth', [6, 8, 10, 12]),
    'iterations': hp.choice('iterations', [500, 1000, 1500]),
    'rsm': hp.uniform('rsm', 0.1, 0.5),
    'subsample': hp.uniform('subsample', 0.4, 0.8),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 5),
    # Additional parameters can be added as needed
}

# Objective function for Bayesian Optimization
def objective(params):
    model = CatBoostRegressor(
        depth=params['depth'],
        iterations=params['iterations'],
        rsm=params['rsm'],
        sampling_frequency="PerTree",
        subsample=params['subsample'],
        cat_features=cat_features_index,  # Replace with the actual index of categorical features
        l2_leaf_reg=params['l2_leaf_reg'],
        verbose=100
    )
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_base, y_base, cv=kf, scoring='neg_mean_squared_error')
    mse_cv = -np.mean(cv_scores)
    return {'loss': mse_cv, 'status': STATUS_OK}

# Initialize the Trials object for storing search results
trials = Trials()

# Run the Bayesian Optimization
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,  # Number of evaluations can be adjusted
            trials=trials)

# Output the best hyperparameters
print("Best hyperparameters:", best)

# Save the best hyperparameters to a CSV file
save_results_to_csv([best], "best_hyperparameters")

  0%|                                                                           | 0/30 [00:00<?, ?trial/s, best loss=?]0:	learn: 1.6376269	total: 39.3ms	remaining: 39.3s
100:	learn: 0.8712853	total: 3.47s	remaining: 30.9s
200:	learn: 0.7532892	total: 6.78s	remaining: 26.9s
300:	learn: 0.7121039	total: 10.4s	remaining: 24.1s
400:	learn: 0.6839139	total: 14.2s	remaining: 21.2s
500:	learn: 0.6652135	total: 17.9s	remaining: 17.8s
600:	learn: 0.6484232	total: 21.5s	remaining: 14.3s
700:	learn: 0.6360793	total: 25.1s	remaining: 10.7s
800:	learn: 0.6259741	total: 28.6s	remaining: 7.11s
900:	learn: 0.6167952	total: 32.2s	remaining: 3.53s
999:	learn: 0.6083658	total: 35.8s	remaining: 0us
0:	learn: 1.6320617	total: 45.5ms	remaining: 45.4s
100:	learn: 0.8718615	total: 3.76s	remaining: 33.4s
200:	learn: 0.7493523	total: 7.74s	remaining: 30.8s
300:	learn: 0.7068992	total: 11.7s	remaining: 27.1s
400:	learn: 0.6809427	total: 15.7s	remaining: 23.5s
500:	learn: 0.6627210	total: 19.7s	remaining: 19.6s
6

300:	learn: 0.6134523	total: 20.3s	remaining: 1m 20s
400:	learn: 0.5811461	total: 27.2s	remaining: 1m 14s
500:	learn: 0.5575481	total: 34.5s	remaining: 1m 8s
600:	learn: 0.5390045	total: 41.5s	remaining: 1m 2s
700:	learn: 0.5270659	total: 48.3s	remaining: 55s
800:	learn: 0.5146504	total: 55.4s	remaining: 48.3s
900:	learn: 0.5039217	total: 1m 2s	remaining: 41.7s
1000:	learn: 0.4956350	total: 1m 9s	remaining: 34.7s
1100:	learn: 0.4889279	total: 1m 16s	remaining: 27.8s
1200:	learn: 0.4828338	total: 1m 23s	remaining: 20.8s
1300:	learn: 0.4775799	total: 1m 30s	remaining: 13.8s
1400:	learn: 0.4730769	total: 1m 37s	remaining: 6.88s
1499:	learn: 0.4691137	total: 1m 45s	remaining: 0us
 10%|████▌                                        | 3/30 [15:04<2:33:14, 340.55s/trial, best loss: 0.21470142803627681]0:	learn: 1.6249816	total: 128ms	remaining: 3m 12s
100:	learn: 0.6934806	total: 10.2s	remaining: 2m 21s
200:	learn: 0.6042614	total: 20.9s	remaining: 2m 15s
300:	learn: 0.5563028	total: 31.3s	rema

200:	learn: 0.5774827	total: 28.4s	remaining: 3m 3s
300:	learn: 0.5305340	total: 42.5s	remaining: 2m 49s
400:	learn: 0.5016302	total: 56.9s	remaining: 2m 36s
500:	learn: 0.4820915	total: 1m 11s	remaining: 2m 23s
600:	learn: 0.4664028	total: 1m 28s	remaining: 2m 12s
700:	learn: 0.4547571	total: 1m 45s	remaining: 2m
800:	learn: 0.4443931	total: 2m 1s	remaining: 1m 45s
900:	learn: 0.4353309	total: 2m 17s	remaining: 1m 31s
1000:	learn: 0.4283571	total: 2m 33s	remaining: 1m 16s
1100:	learn: 0.4224338	total: 2m 50s	remaining: 1m 1s
1200:	learn: 0.4172508	total: 3m 6s	remaining: 46.4s
1300:	learn: 0.4125592	total: 3m 23s	remaining: 31.1s
1400:	learn: 0.4083113	total: 3m 39s	remaining: 15.5s
1499:	learn: 0.4041309	total: 3m 56s	remaining: 0us
0:	learn: 1.6236494	total: 194ms	remaining: 4m 50s
100:	learn: 0.6732504	total: 16.7s	remaining: 3m 51s
200:	learn: 0.5809125	total: 34.7s	remaining: 3m 44s
300:	learn: 0.5334209	total: 53.1s	remaining: 3m 31s
400:	learn: 0.5029798	total: 1m 10s	remaining

1100:	learn: 0.4801045	total: 1m 41s	remaining: 36.8s
1200:	learn: 0.4743313	total: 1m 49s	remaining: 27.4s
1300:	learn: 0.4688269	total: 1m 58s	remaining: 18.2s
1400:	learn: 0.4645060	total: 2m 6s	remaining: 8.95s
1499:	learn: 0.4604051	total: 2m 14s	remaining: 0us
 23%|██████████                                 | 7/30 [1:08:10<4:47:56, 751.14s/trial, best loss: 0.17632722688117602]0:	learn: 1.6321920	total: 43.2ms	remaining: 21.5s
100:	learn: 0.7471333	total: 6.24s	remaining: 24.6s
200:	learn: 0.6561835	total: 13.6s	remaining: 20.2s
300:	learn: 0.6124434	total: 20.2s	remaining: 13.3s
400:	learn: 0.5859800	total: 27.6s	remaining: 6.8s
499:	learn: 0.5645322	total: 36.5s	remaining: 0us
0:	learn: 1.6308330	total: 75.2ms	remaining: 37.5s
100:	learn: 0.7507774	total: 6.85s	remaining: 27.1s
200:	learn: 0.6606468	total: 13.6s	remaining: 20.2s
300:	learn: 0.6226358	total: 19.6s	remaining: 12.9s
400:	learn: 0.5915907	total: 26.1s	remaining: 6.44s
499:	learn: 0.5687849	total: 32.2s	remaining: 0

## Model 2): XGboost

In [41]:
# Initialize XGBoost Regressor with new hyperparameters
xgb_model = XGBRegressor(
    learning_rate=0.05,
    max_depth=10,
    min_child_weight=5,
    n_estimators=1000,
    n_jobs=-1,
    objective='reg:squarederror',
    subsample=0.8,
    colsample_bytree=0.7,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=0.1,
    verbosity=0
)

# Create XGBoost pipeline with preprocessor and model
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])

# Fit the pipeline to the base training data
xgb_pipeline.fit(X_base, y_base)

# Perform K-Fold Cross-Validation with RMSE scoring
xgb_cv_scores = cross_val_score(xgb_pipeline, X_base, y_base, cv=kf, scoring='neg_root_mean_squared_error')

# Calculate positive RMSE scores
xgb_cv_rmse_scores = -xgb_cv_scores

# Display RMSE scores and average RMSE
print("XGBoost K-Fold Cross-Validation RMSE Scores:", xgb_cv_rmse_scores)
print("Average RMSE:", np.mean(xgb_cv_rmse_scores))

# Extract feature importances from XGBoost model
xgb_feature_importances = xgb_pipeline.named_steps['model'].feature_importances_

# Display feature importances
print("XGBoost Feature Importances:")
for importance, feature in sorted(zip(xgb_feature_importances, preprocessor.get_feature_names_out()), reverse=True):
    print(f"{feature}: {importance:.4f}")

XGBoost K-Fold Cross-Validation RMSE Scores: [0.78811683 0.8007234  0.79043866 0.78682469 0.79026674]
Average RMSE: 0.7912740636315023
XGBoost Feature Importances:
cat__counter_name_Totem 73 boulevard de Sébastopol S-N: 0.1223
cat__counter_name_Totem 73 boulevard de Sébastopol N-S: 0.1020
cat__counter_name_Face au 40 quai D'Issy SO-NE: 0.0764
cat__counter_name_Face au 40 quai D'Issy NE-SO: 0.0547
cat__counter_name_28 boulevard Diderot E-O: 0.0542
cat__counter_name_67 boulevard Voltaire SE-NO: 0.0513
cat__counter_name_Totem 64 Rue de Rivoli O-E: 0.0494
cat__counter_name_27 quai de la Tournelle SE-NO: 0.0426
cat__counter_name_Face au 8 avenue de la porte de Charenton NO-SE: 0.0289
cat__counter_name_Totem 85 quai d'Austerlitz SE-NO: 0.0273
cat__counter_name_Totem 64 Rue de Rivoli E-O: 0.0261
cat__counter_name_Face au 48 quai de la marne SO-NE: 0.0257
cat__counter_name_Pont des Invalides S-N: 0.0218
cat__counter_name_Totem Cours la Reine O-E: 0.0204
cat__counter_name_Face au 48 quai de la 

In [65]:
# Define the parameter grid for Random Search
param_dist = {
    'model__learning_rate': np.linspace(0.01, 0.1, 10),
    'model__max_depth': range(3, 10),
    'model__min_child_weight': range(1, 10),
    'model__n_estimators': range(100, 1000, 100),
    'model__subsample': np.linspace(0.5, 1, 6),
    'model__colsample_bytree': np.linspace(0.5, 1, 6),
    'model__gamma': np.linspace(0, 0.5, 6),
    'model__reg_alpha': np.linspace(0, 0.5, 6),
    'model__reg_lambda': np.linspace(0, 0.5, 6)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(xgb_pipeline, param_distributions=param_dist, n_iter=50, scoring='neg_root_mean_squared_error', cv=kf, verbose=2, n_jobs=-1, random_state=42)

# Perform Random Search
random_search.fit(X_base, y_base)

# Output the best parameters and corresponding RMSE score
best_params = random_search.best_params_
best_rmse = -ranom_search.best_score_

# Print best parameters and RMSE score
print("Best parameters found: ", best_params)
print("Best RMSE score: ", best_rmse)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'model__subsample': 0.7, 'model__reg_lambda': 0.0, 'model__reg_alpha': 0.0, 'model__n_estimators': 600, 'model__min_child_weight': 5, 'model__max_depth': 9, 'model__learning_rate': 0.1, 'model__gamma': 0.5, 'model__colsample_bytree': 1.0}
Best RMSE score:  0.729969706985908


In [ ]:
best_results = pd.DataFrame([best_params])
best_results['best_rmse'] = best_rmse
save_results_to_csv(best results_, 'Grid_search_results_XGBoost')

In [ ]:
# Grid search parameters defined based on the results of Random Search
param_grid = {
    'model__learning_rate': [0.05, 0.1, 0.15],
    'model__max_depth': [7, 8, 9, 10],
    'model__min_child_weight': [4, 5, 6],
    'model__n_estimators': [550, 600, 650],
    'model__subsample': [0.65, 0.7, 0.75],
    'model__colsample_bytree': [0.95, 1.0, 1.05],
    'model__gamma': [0.45, 0.5, 0.55],
    'model__reg_alpha': [0.0, 0.05, 0.1],
    'model__reg_lambda': [0.0, 0.05, 0.1]
}

# Initialize GridSearchCV with the XGBoost pipeline
grid_search = GridSearchCV(xgb_pipeline, param_grid, scoring='neg_root_mean_squared_error', cv=kf, n_jobs=-1, verbose=2)

# Perform the Grid Search
grid_search.fit(X_base, y_base)

# Extracting Grid Search results into a DataFrame
grid_search_results = grid_search.cv_results_

save_results_to_csv(results_random_search_xgb, 'grid_search_results_XGBoost')
# Output the best parameters and corresponding RMSE score
best_params = grid_search.best_params_
best_rmse = -grid_search.best_score_

print("Best parameters found: ", best_params)
print("Best RMSE score: ", best_rmse)
print(f"Grid Search results saved as {filename}")


Fitting 5 folds for each of 26244 candidates, totalling 131220 fits


In [78]:
best_results = pd.DataFrame([best_params])
best_results['best_rmse'] = best_rmse
save_results_to_csv(best results_, 'Grid_search_results_XGBoost')

Results saved as random_search_results_XGBoost_20231209_071055.csv


## Model 3) Ridge regression

In [43]:
# Initialize Ridge Regression with placeholders for hyperparameters
ridge_model = Ridge(alpha=0.9850000000000002)


# Create Ridge Regression pipeline
ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', ridge_model)
])

# Fit the Ridge Regression pipeline to the entire dataset
ridge_pipeline.fit(X_base, y_base)

# Perform K-Fold Cross-Validation with RMSE scoring
ridge_cv_scores = cross_val_score(ridge_pipeline, X_base, y_base, cv=kf, scoring='neg_root_mean_squared_error')

# Convert negative RMSE scores to positive
ridge_cv_rmse_scores = -ridge_cv_scores

# Display RMSE scores and average RMSE
print("Ridge Regression K-Fold Cross-Validation RMSE Scores:", ridge_cv_rmse_scores)
print("Average RMSE:", np.mean(ridge_cv_rmse_scores))

Ridge Regression K-Fold Cross-Validation RMSE Scores: [1.46140368 1.46430465 1.45831644 1.46140571 1.46132465]
Average RMSE: 1.4613510259043003


In [ ]:
# Define the grid of hyperparameters to search
param_grid = {
    'model__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]  # A range of values for alpha
}

# Initialize GridSearchCV
grid_search = GridSearchCV(ridge_pipeline, param_grid, cv=kf, scoring='neg_root_mean_squared_error', verbose=1)

# Fit GridSearchCV
grid_search.fit(X_base, y_base)

# Output the best parameters and the corresponding RMSE
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print("Best Parameters:", best_params)
print("Best RMSE:", best_score)

In [ ]:
best_results = pd.DataFrame([best_params])
best_results['best_rmse'] = best_rmse
save_results_to_csv(best results_, 'Grid_search_results_XGBoost')

In [44]:
# Extract feature names after preprocessing
feature_names_transformed = ridge_pipeline.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out()

# Combine with the remaining feature names
feature_names = np.concatenate((feature_names_transformed, numerical_features))

# Extract the model's coefficients
ridge_coefficients = ridge_pipeline.named_steps['model'].coef_

# Display the feature importances (coefficients)
print("Ridge Regression Feature Importances:")
for coef, feature in sorted(zip(ridge_coefficients, feature_names), key=lambda x: abs(x[0]), reverse=True):
    print(f"{feature}: {coef:.4f}")


Ridge Regression Feature Importances:
humidity: -0.4986
humidity: -0.4227
SNCFpassengersDelayedInParisPerHour: -0.3293
windspeed: -0.3170
Subscribers: -0.2183
precip: -0.2106
Subscribers: 0.1369
precip: 0.0553
windspeed: 0.0528
SNCFpassengersDelayedInParisPerHour: 0.0016


## Stacking:

In [45]:
# Predictions from CatBoost model
catboost_predictions = catboost_model.predict(X_meta)

# Predictions from XGBoost model
xgb_predictions = xgb_pipeline.predict(X_meta)

# Predictions from Ridge Regression model
ridge_predictions = ridge_pipeline.predict(X_meta)

In [46]:
# Stacking predictions
stacked_predictions = np.column_stack((catboost_predictions, xgb_predictions, ridge_predictions))

In [47]:
# Initialize and fit the OLS model
ols_model = LinearRegression()
ols_model.fit(stacked_predictions, y_meta)

LinearRegression()

In [48]:
# Define a function to evaluate the stacked model
def evaluate_stacked_model(X, y, kf, base_models, final_model):
    rmse_scores = []
    for train_index, test_index in kf.split(X):
        # Split data
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Get base model predictions
        base_predictions = [model.predict(X_test) for model in base_models]

        # Stack predictions
        stacked_predictions = np.column_stack(base_predictions)

        # Predict with final model
        final_pred = final_model.predict(stacked_predictions)

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_test, final_pred))
        rmse_scores.append(rmse)

    return rmse_scores

# Base models list
base_models = [catboost_model, xgb_pipeline, ridge_pipeline]

# Evaluate the stacked model
stacked_rmse_scores = evaluate_stacked_model(X, y, kf, base_models, ols_model)
print("Stacked Model RMSE Scores:", stacked_rmse_scores)
print("Average RMSE:", np.mean(stacked_rmse_scores))


Stacked Model RMSE Scores: [0.45921663431578363, 0.45405431442895056, 0.45030573773283716, 0.4500850979286849, 0.4504576855392574]
Average RMSE: 0.4528238939891027


In [49]:
# Extract the weights (coefficients) from the OLS model
ols_weights = ols_model.coef_

# Display the weights for each base model
print("Weights assigned to each base model's predictions by OLS regression:")
base_model_names = ['CatBoost', 'XGBoost', 'Ridge Regression']
for model_name, weight in zip(base_model_names, ols_weights):
    print(f"{model_name}: {weight:.4f}")


Weights assigned to each base model's predictions by OLS regression:
CatBoost: 1.0137
XGBoost: 0.0195
Ridge Regression: -0.0244


### Retraining the models

In [51]:
# Retrain CatBoost model on entire training data
catboost_model.fit(X, y)

# Retrain XGBoost model on entire training data
xgb_pipeline.fit(X, y)

# Retrain Ridge Regression model on entire training data
ridge_pipeline.fit(X, y)

0:	learn: 1.6335061	total: 160ms	remaining: 2m 39s
100:	learn: 0.7013328	total: 16.4s	remaining: 2m 26s
200:	learn: 0.6133109	total: 33.2s	remaining: 2m 12s
300:	learn: 0.5674265	total: 50.2s	remaining: 1m 56s
400:	learn: 0.5338198	total: 1m 8s	remaining: 1m 41s
500:	learn: 0.5149646	total: 1m 25s	remaining: 1m 25s
600:	learn: 0.4975595	total: 1m 43s	remaining: 1m 8s
700:	learn: 0.4860409	total: 1m 59s	remaining: 51.1s
800:	learn: 0.4756117	total: 2m 14s	remaining: 33.5s
900:	learn: 0.4675631	total: 2m 32s	remaining: 16.8s
999:	learn: 0.4612285	total: 2m 49s	remaining: 0us


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Subscribers',
                                                   'SNCFpassengersDelayedInParisPerHour',
                                                   'humidity', 'precip',
                                                   'windspeed']),
                                                 ('cat', OneHotEncoder(),
                                                  ['counter_name'])])),
                ('model', Ridge(alpha=0.9850000000000002))])

In [52]:
# Predictions from CatBoost model
catboost_test_predictions = catboost_model.predict(test_data_processed)

# Predictions from XGBoost model
xgb_test_predictions = xgb_pipeline.predict(test_data_processed)

# Predictions from Ridge Regression model
ridge_test_predictions = ridge_pipeline.predict(test_data_processed)

In [53]:
# Stack predictions
stacked_test_predictions = np.column_stack((catboost_test_predictions, xgb_test_predictions, ridge_test_predictions))
# Final predictions using OLS model
final_predictions = ols_model.predict(stacked_test_predictions)

In [ ]:
# Saving my best model results
save_submission_csv(test_data_processed, catboost_test_predictions, "CatboostModel")

In [54]:
# Saving the final stacked model results
save_submission_csv(test_data_processed, final_predictions, "StackedModel")


          Id  log_bike_count
0          0        0.973225
1          1        1.576168
2          2        1.509435
3          3        0.972645
4          4        1.240719
...      ...             ...
41603  41603        5.985046
41604  41604        4.921817
41605  41605        5.317443
41606  41606        3.387701
41607  41607        2.405477

[41608 rows x 2 columns]
